# 21. Implement the MT19937 Mersenne Twister RNG

https://cryptopals.com/sets/3/challenges/21

https://en.wikipedia.org/wiki/Mersenne_Twister

In [1]:
class MT19937:

    # Coefficients for MT19937
    (w, n, m, r) = (32, 624, 397, 31)
    a = 0x9908B0DF
    (u, d) = (11, 0xFFFFFFFF)
    (s, b) = (7, 0x9D2C5680)
    (t, c) = (15, 0xEFC60000)
    l = 18
    f = 1812433253

    UMASK = 0xFFFFFFFF & (0xFFFFFFFF << r)       # Limit to 32 bits
    LMASK = 0xFFFFFFFF & (0xFFFFFFFF >> (w - r)) # Limit to 32 bits

    def __init__(self, seed: int = 19650218):
        self.state_array = [0] * self.n  # Array for the state vector
        self.state_index = 0            # Index into state vector array, 0 <= state_index <= n-1
        self.initialize_state(seed)

    def initialize_state(self, seed: int = 19650218):
        self.state_array[0] = seed & 0xFFFFFFFF  # Limit to 32 bits
        for i in range(1, self.n):
            seed = (self.f * (seed ^ (seed >> (self.w - 2))) + i) & 0xFFFFFFFF # Limit to 32 bits
            self.state_array[i] = seed
        self.state_index = 0

    def random(self):
        k = self.state_index               # Current state location
        j = (k - (self.n - 1)) % self.n    # n-1 iterations before

        x = (self.state_array[k] & self.UMASK) | (self.state_array[j] & self.LMASK)
        xA = x >> 1
        if x & 0x00000001:
            xA ^= self.a

        j = (k - (self.n - self.m)) % self.n  # n-m iterations before
        x = self.state_array[j] ^ xA          # Compute next value in the state

        self.state_array[k] = x & 0xFFFFFFFF  # Limit to 32 bits
        k = (k + 1) % self.n                  # Circular indexing
        self.state_index = k

        # Tempering
        y = x ^ (x >> self.u)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        z = y ^ (y >> self.l)
        return z & 0xFFFFFFFF  # Return 32-bit integer

In [2]:
mt_random = MT19937()

for _ in range(10):
    print(mt_random.random())

2325592414
482149846
4177211283
3872387439
1663027210
2005191859
666881213
3289399202
2514534568
3882134983


# 22. Crack an MT19937 seed

https://cryptopals.com/sets/3/challenges/22

In [11]:
import time
import random

def get_MT19937_int32(reallywait=True,tmin=40,tmax=1000):
    '''
    Wait a random number of seconds between 40 and 1000
    Seeds the RNG with the current Unix timestamp
    Waits a random number of seconds again.
    Returns the first 32 bit output of the RNG.
    '''
    sleep1 = random.uniform(tmin,tmax)
    sleep2 = random.uniform(tmin,tmax)
    print(f"Waiting {sleep1:2.2f} seconds...")
    if reallywait:
        time.sleep(sleep1)
        seed = int(time.time())
    else:
        seed = int(time.time()-sleep1-sleep2)
    print(f"Seedind MT19937 with current Unix time ({seed})...")
    mt_random = MT19937(seed)
    sleep = random.uniform(tmin,tmax)
    print(f"Waiting {sleep:2.2f} seconds...")
    if reallywait:
        time.sleep(sleep2)
    rnd32 = mt_random.random()
    print(f"Serving random number {rnd32}")
    return rnd32

def crack_MT19937_seed(rnd32):
    print("Attempting to guess MT19937 seed...")
    now = int(time.time())+1
    while True:
        mt_random = MT19937(now)
        _rnd = mt_random.random()
        if rnd32==_rnd:
            print(f"Guessed seed: {now}")
            return now
        # go back in time
        now -= 1

In [12]:
rnd32 = get_MT19937_int32(False)
print()
crack_MT19937_seed(rnd32)

Waiting 651.03 seconds...
Seedind MT19937 with current Unix time (1736598814)...
Waiting 745.50 seconds...
Serving random number 3274087935

Attempting to guess MT19937 seed...
Guessed seed: 1736598814


1736598814

In [13]:
rnd32 = get_MT19937_int32(True,40,100)
print()
crack_MT19937_seed(rnd32)

Waiting 64.02 seconds...
Seedind MT19937 with current Unix time (1736600389)...
Waiting 77.19 seconds...
Serving random number 3880950902

Attempting to guess MT19937 seed...
Guessed seed: 1736600389


1736600389